# Portraits en nouances 
A study on portraits through the modern age and with focus on quantitative information and a color scheme analysis

In [ ]:
import pandas as pd 
import pprint 
from pprint import pprint
import matplotlib.pyplot as plt


The query is posted here for future repruducibility, but the querying process has been performed directly on the [Wikidata Query Service](https://query.wikidata.org/), and the results downloaded in a .csv format and then reused in the analysis section. 

In [ ]:
#query on wdt qs: 
'''SELECT DISTINCT?painting ?title ?img ?inceptionYear ?name ?genderLabel ?countryLabel ?movementLabel WHERE {
  ?painting wdt:P31 wd:Q3305213;
    wdt:P136 wd:Q134307;
    rdfs:label ?title;
    wdt:P571 ?inceptionYear;
    wdt:P170 ?author;
    wdt:P18 ?img.
  ?author rdfs:label ?name;
          wdt:P21 ?gender;
          wdt:P27 ?country;
          wdt:P135 ?movement.
  ?gender rdfs:label ?genderLabel.
  OPTIONAL{?movement rdfs:label ?movementLabel.}
  ?country rdfs:label ?countryLabel.
  FILTER(("1869-01-01"^^xsd:dateTime <= ?inceptionYear) && (?inceptionYear < "1950-01-01"^^xsd:dateTime))
  FILTER((LANG(?title)) = "en")
  FILTER((LANG(?name)) = 'en')
  FILTER((LANG(?genderLabel)) = 'en')
  FILTER((LANG(?movementLabel)) = 'en')
  FILTER((LANG(?countryLabel)) = 'en')
}'''

'SELECT ?painting ?title ?img ?inceptionYear ?name ?genderLabel ?countryLabel ?movementLabel WHERE {\n  ?painting wdt:P31 wd:Q3305213;\n    wdt:P136 wd:Q134307;\n    rdfs:label ?title;\n    wdt:P571 ?inceptionYear;\n    wdt:P170 ?author;\n    wdt:P18 ?img.\n  ?author rdfs:label ?name;\n          wdt:P21 ?gender;\n          wdt:P27 ?country;\n          wdt:P135 ?movement.\n  ?gender rdfs:label ?genderLabel.\n  OPTIONAL{?movement rdfs:label ?movementLabel.}\n  ?country rdfs:label ?countryLabel.\n  FILTER(("1869-01-01"^^xsd:dateTime <= ?inceptionYear) && (?inceptionYear < "1950-01-01"^^xsd:dateTime))\n  FILTER((LANG(?title)) = "en")\n  FILTER((LANG(?name)) = \'en\')\n  FILTER((LANG(?genderLabel)) = \'en\')\n  FILTER((LANG(?movementLabel)) = \'en\')\n  FILTER((LANG(?countryLabel)) = \'en\')\n}'

# IMPORT AND EXPLORE DATASET

In [ ]:
with open('/content/drive/MyDrive/QUERY_BIG.csv') as f: 
    data = pd.read_csv(f)

In [ ]:
data.describe()

,painting,title,img,inceptionYear,name,genderLabel,countryLabel,movementLabel
count,6973,6973,6973,6973,6973,6973,6973,6973
unique,3726,3402,3960,115,453,2,80,114
top,http://www.wikidata.org/entity/Q23688779,Portrait of a Woman,http://commons.wikimedia.org/wiki/Special:File...,1890-01-01T00:00:00Z,Edvard Munch,male,France,Impressionism
freq,16,55,15,234,356,6728,1595,1584


Remove duplicates in the dataset (some duplicates due to multiple values in the nationality of the author)

In [ ]:
data.drop_duplicates(subset = 'painting', inplace = True)

In [ ]:
data.describe()

,painting,title,img,inceptionYear,name,genderLabel,countryLabel,movementLabel
count,3726,3726,3726,3726,3726,3726,3726,3726
unique,3726,3402,3726,113,453,2,65,84
top,http://www.wikidata.org/entity/Q17495363,Portrait of a Man,http://commons.wikimedia.org/wiki/Special:File...,1890-01-01T00:00:00Z,Pierre-Auguste Renoir,male,France,Impressionism
freq,1,26,1,131,171,3544,847,1037


# DATA CLEANING 

*  **Geographical unification:** This step is merging each country specific name or period with its current state/geographical area. Es: "russian empire" --> "Russia", "Cisleithania" --> "Austria".
*   **Movement unification"** This step merges occurrences of the same artistic movements that are too specific (es: "Amsterdam Impressionism" --> "Impressionism") and also cases where the same artistic movement has been written in slightly different forms (es: "realism" --> "Realism"). 



In [ ]:
#GEO MERGING 
data.loc[data['countryLabel'] == 'Russian Empire', 'countryLabel'] = 'Russia'
data.loc[data['countryLabel'] == 'Soviet Union', 'countryLabel'] = 'Russia'
data.loc[data['countryLabel'] == 'United States', 'countryLabel'] = 'USA'
data.loc[data['countryLabel'] == 'United States of America', 'countryLabel'] = 'USA'
data.loc[data['countryLabel'] == 'Kingdom of the Netherlands', 'countryLabel'] = 'Netherlands'
data.loc[data['countryLabel'] == 'Kingdom of Italy', 'countryLabel'] = 'Italy'
data.loc[data['countryLabel'] == 'Kingdom of Prussia', 'countryLabel'] = 'Germany'
data.loc[data['countryLabel'] == 'German Reich', 'countryLabel'] = 'Germany'
data.loc[data['countryLabel'] == 'Cisleithania', 'countryLabel'] = 'Austria'
data.loc[data['countryLabel'] == 'Austrian Empire', 'countryLabel'] = 'Austria'
data.loc[data['countryLabel'] == 'Austria-Hungary', 'countryLabel'] = 'Austria'
data.loc[data['countryLabel'] == 'Second Polish Republic', 'countryLabel'] = 'Poland'
data.loc[data['countryLabel'] == 'United Kingdom of Great Britain and Ireland', 'countryLabel'] = 'United Kingdom'
data.loc[data['countryLabel'] == 'Grand Duchy of Finland', 'countryLabel'] = 'Finland'
data.loc[data['countryLabel'] == 'Kingdom of Hungary', 'countryLabel'] = 'Hungary'
data.loc[data['countryLabel'] == 'French Second Republic', 'countryLabel'] = 'France'
data.loc[data['countryLabel'] == 'German Empire', 'countryLabel'] = 'Germany'
data.loc[data['countryLabel'] == 'Russian Soviet Federative Socialist Republic', 'countryLabel'] = 'Russia'
data.loc[data['countryLabel'] == 'Congress Poland', 'countryLabel'] = 'Poland'
data.loc[data['countryLabel'] == 'Weimar Republic', 'countryLabel'] = 'Germany'
data.loc[data['countryLabel'] == 'French Third Republic', 'countryLabel'] = 'France'
data.loc[data['countryLabel'] == 'Kingdom of Denmark', 'countryLabel'] = 'Denmark'
data.loc[data['countryLabel'] == 'Republic of Ireland', 'countryLabel'] = 'Ireland'
data.loc[data['countryLabel'] == 'Great Britain', 'countryLabel'] = 'United Kingdom'
data.loc[data['countryLabel'] == 'Wales', 'countryLabel'] = 'United Kingdom'
data.loc[data['countryLabel'] == 'Spanish Netherlands', 'countryLabel'] = 'Netherlands'
data.loc[data['countryLabel'] == 'Southern Netherlands', 'countryLabel'] = 'Netherlands'
data.loc[data['countryLabel'] == 'Republic of Venice', 'countryLabel'] = 'Italy'
data.loc[data['countryLabel'] == 'Ottoman Empire', 'countryLabel'] = 'Turkey'
data.loc[data['countryLabel'] == 'Nazi Germany', 'countryLabel'] = 'Germany'
data.loc[data['countryLabel'] == 'Grand Duchy of Tuscany', 'countryLabel'] = 'Italy'
data.loc[data['countryLabel'] == 'Empire of Japan', 'countryLabel'] = 'Japan'
data.loc[data['countryLabel'] == 'Dutch Republic', 'countryLabel'] = 'Netherlands'
data.loc[data['countryLabel'] == 'British Raj', 'countryLabel'] = 'India'

In [ ]:
#MOVEMENT MERGING 
data.loc[data['movementLabel'] == 'realism', 'movementLabel'] = 'Realism'
data.loc[data['movementLabel'] == 'Barbizon school', 'movementLabel'] = 'Realism'
data.loc[data['movementLabel'] == 'socialist realism', 'movementLabel'] = 'Realism'
data.loc[data['movementLabel'] == 'Hague School', 'movementLabel'] = 'Realism'
data.loc[data['movementLabel'] == 'American realism', 'movementLabel'] = 'Realism'
data.loc[data['movementLabel'] == 'social realism', 'movementLabel'] = 'Realism'
data.loc[data['movementLabel'] == 'Naturalism', 'movementLabel'] = 'Realism'
data.loc[data['movementLabel'] == 'neo-impressionism', 'movementLabel'] = 'Neo-Impressionism'
data.loc[data['movementLabel'] == 'pointillism', 'movementLabel'] = 'Neo-Impressionism'
data.loc[data['movementLabel'] == 'fauvism', 'movementLabel'] = 'Post-Impressionism'
data.loc[data['movementLabel'] == 'pointilism', 'movementLabel'] = 'Post-Impressionism'
data.loc[data['movementLabel'] == 'expressionism', 'movementLabel'] = 'Expressionism'
data.loc[data['movementLabel'] == 'post-impressionism', 'movementLabel'] = 'Post-Impressionism'
data.loc[data['movementLabel'] == 'Japonisme', 'movementLabel'] = 'Impressionism'
data.loc[data['movementLabel'] == 'Heidelberg School', 'movementLabel'] = 'Impressionism'
data.loc[data['movementLabel'] == 'Amsterdam Impressionism', 'movementLabel'] = 'Impressionism'
data.loc[data['movementLabel'] == 'bohemianism', 'movementLabel'] = 'Impressionism'
data.loc[data['movementLabel'] == 'primitivism', 'movementLabel'] = 'Post-Impressionism'
data.loc[data['movementLabel'] == 'Cloisonnism', 'movementLabel'] = 'Post-Impressionism'
data.loc[data['movementLabel'] == 'naïve art', 'movementLabel'] = 'Post-Impressionism'
data.loc[data['movementLabel'] == 'analytical cubism', 'movementLabel'] = 'Cubism'
data.loc[data['movementLabel'] == 'Pont-Aven School', 'movementLabel'] = 'Post-Impressionism'
data.loc[data['movementLabel'] == 'Les Nabis', 'movementLabel'] = 'Post-Impressionism'
data.loc[data['movementLabel'] == 'constructivism', 'movementLabel'] = 'Futurism'
data.loc[data['movementLabel'] == 'naïve art', 'movementLabel'] = 'Post-Impressionism'
data.loc[data['movementLabel'] == 'symbolism', 'movementLabel'] = 'Symbolism'
data.loc[data['movementLabel'] == 'Russian symbolism', 'movementLabel'] = 'Symbolism'
data.loc[data['movementLabel'] == 'degenerate art', 'movementLabel'] = 'Modernism'
data.loc[data['movementLabel'] == 'modern art', 'movementLabel'] = 'Modernism'
data.loc[data['movementLabel'] == 'Dada', 'movementLabel'] = 'Modernism'
data.loc[data['movementLabel'] == 'Pleinairisme', 'movementLabel'] = 'Impressionism'
data.loc[data['movementLabel'] == 'New Objectivity', 'movementLabel'] = 'Expressionism'
data.loc[data['movementLabel'] == 'modernism', 'movementLabel'] = 'Modernism'
data.loc[data['movementLabel'] == 'Catalan modernism', 'movementLabel'] = 'Modernism'
data.loc[data['movementLabel'] == 'Young Poland', 'movementLabel'] = 'Modernism'
data.loc[data['movementLabel'] == 'German Expressionism', 'movementLabel'] = 'Expressionism'
data.loc[data['movementLabel'] == 'Glasgow School', 'movementLabel'] = 'Art Nouveau'
data.loc[data['movementLabel'] == 'academic art', 'movementLabel'] = 'Academic Art'
data.loc[data['movementLabel'] == 'Pre-Raphaelite Brotherhood', 'movementLabel'] = 'Academic Art'
data.loc[data['movementLabel'] == 'futurism', 'movementLabel'] = 'Futurism'
data.loc[data['movementLabel'] == 'American Impressionism', 'movementLabel'] = 'Impressionism'


# EXTRACT RELEVANT INFORMATION FROM THE DATASET
In this section we explore some of the insights our dataset can provide, such as: 


*   portraits per country
*   painters per country 
* gender division of paintings according to their author's gender 
* painters per movement 
* portraits per movement 
* top 15 movements per number of portraits 
* top 15 painters per paintings and respective movement 
* movements per country 

All these information extracted and sysntesised from our data will be useful for the storytelling section in our website, since they provide relevant quantitative information on the subject matter. 



In [ ]:
#NUMBER OF PORTRAITS PER COUNTRY
portraits_per_country = data.groupby('countryLabel')['painting'].count()
portraits_per_country.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/N_portraits_per_country.csv', header = False)

In [ ]:
#NUMBER OF PAINTERS PER COUNTRY
painters_per_country = data.groupby('countryLabel')['name'].nunique()
painters_per_country.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/N_painters_per_country.csv', header = False)

In [ ]:
#NUMBER OF PAINTERS PER GENDER
painters_per_gender = data.groupby('genderLabel')['name'].nunique()
painters_per_gender.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/N_painters_per_gender.csv', header = False)

In [ ]:
#NUMBER OF PAINTERS PER MOVEMENT 
painters_per_movement = data.groupby('movementLabel')['name'].nunique()
painters_per_movement.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/N_painters_per_movement.csv', header = False)

In [ ]:
#NUMBER OF PORTRAITS PER MOVEMENT 
portraits_per_movement = data.groupby('movementLabel')['painting'].count()
portraits_per_movement.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/N_portraits_per_movement.csv', header = False)

In [ ]:
#TOP 15 MOVEMENTS PER NUMBER OF PAINTINGS 
top_movements = data.value_counts(subset = 'movementLabel')
top_movements = top_movements[0:15]
top_movements.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/TOP_15_MOVEMENTS_BY_PAINTING.csv', header = False)

In [ ]:
#TOP 15 PAINTERS FOR PAINTINGS WITH RESPECTIVE MOVEMENT 
top_15_painters_and_movement = data.groupby('name')['movementLabel'].value_counts().nlargest(15)
top_15_painters_and_movement.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/TOP_15_PAINTERS_AND_THEIR_MOVEMENT.csv', header = False)

In [ ]:
#MOVEMENTS PER COUNTRY (WHICH MOVEMENTS IN EACH COUNTRY) AS A LIST (COUNTRY --> [MOV,MOV,MOV,MOV...])
movements_per_country = data.groupby('countryLabel')['movementLabel'].unique()
movements_per_country = pd.DataFrame(movements_per_country) 
movements_per_country.movementLabel.apply(pd.Series).add_prefix('movement_')
movements_per_country.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/which_movements_per_country_AS_LIST.csv')

In [ ]:
#MOVEMENTS PER COUNTRY AS CSV (COUNTRY MOV MOV MOV...)
new = movements_per_country.movementLabel.apply(pd.Series).add_prefix('movement_')
new.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/which_movements_per_country.csv')

# DOWNLOAD THE IMAGES FOR THE COLOR ANALYSIS 
This step has been performed just once and it downloads the images of the paintings from their Wikidata URLs to store them into separated folders ready for the color analysis. 

In [ ]:
img_impressionism = data.loc[data['movementLabel'] == 'Impressionism', ['img']]
img_realism = data.loc[data['movementLabel'] == 'Realism', ['img']]
img_expressionism = data.loc[data['movementLabel'] == 'Expressionism', ['img']]
img_symbolism = data.loc[data['movementLabel'] == 'Symbolism', ['img']]
img_postImp = data.loc[data['movementLabel'] == 'Post-Impressionism', ['img']]
img_academicArt = data.loc[data['movementLabel'] == 'Academic Art', ['img']]
img_modernism = data.loc[data['movementLabel'] == 'Modernism', ['img']]
img_orientalism = data.loc[data['movementLabel'] == 'Orientalism', ['img']]
img_ArtNouveau = data.loc[data['movementLabel'] == 'Art Nouveau', ['img']]
img_romanticism = data.loc[data['movementLabel'] == 'Romanticism', ['img']]
img_NeoImpressionism = data.loc[data['movementLabel'] == 'Neo-Impressionism', ['img']]
img_cubism = data.loc[data['movementLabel'] == 'Cubism', ['img']]

In [ ]:
import requests
session = requests.session()
session.headers

{'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

This function downloads the images of our dataset from their URL and organizes them into separate folders that will be used in the color analysis further on. 

In [ ]:
from pandas.core.groupby import base
import base64 
def download(table, dir_name): 
    uri_list = table['img'].tolist()
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36'}
    for index, uri in enumerate(uri_list):
        filename = '/content/drive/MyDrive/PAINTINGS_BIG/' + dir_name + str(index) + '.jpg'
        with open(filename, 'wb') as handle: 
            response = requests.get(uri, stream = True, headers = headers)
            decoded = base64.b64decode(response.content)
            #print(response)
            for block in response.iter_content(1024): 
                if not block: 
                    break
                handle.write(block)

In [ ]:
download(img_academicArt, 'academicArt')

In [ ]:
download(img_postImp, 'postImp')

In [ ]:
download(img_impressionism, 'impressionism')

In [ ]:
download(img_realism, 'realism')

In [ ]:
download(img_ArtNouveau, 'artNouveau')

In [ ]:
download(img_NeoImpressionism, 'neoImpr')

In [ ]:
download(img_expressionism, 'expressionism')

In [ ]:
download(img_modernism, 'modernism')

In [ ]:
download(img_orientalism, 'orientalism')

In [ ]:
download(img_symbolism, 'symbolism')

In [ ]:
download(img_cubism, 'cubism')

# COLOR ANALYSIS: PALETTE EXTRACTION 

In this part, the previously downloaded pictures are analysed in order to extract the most prominent colors and create a palette for each artistic movement. 
These are the 3 functions composing the pipeline of the palette extraction :


*   **extract_rgb_palette**: this function uses a module of the python library [Color Thief](https://github.com/fengsp/color-thief-py) to extract the k most prominent rgb values from a given image. The function returns a pandas dataframe in which each image (row) has 6 rgb values (columns) in the form of a tuple. This function is applied to the whole set of images that refer to a certain movement, thus creating a big dataframe which contain color palette information for each portrait in that artistic movement. 
*  **extract_names_palette**: This function takes in input the pandas dataframe from the previous function and converts the whole dataframe into a dataframe composed of the CSS webcolors equivalent of the rgb palette dataframe. This step is crucial, and this is provided by the ancillary function
    * **convert_rgb_to_names** function: this function uses The [*KDTree alorithm* as implemented in Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html) to convert each rgb value to the nearest CSS color name contained in the [webcolors](https://pypi.org/project/webcolors/) css_to_names dictionary. Since the original rgb values extracted are too sparse to make any reasonable comparison and analysis among them, this step is necessary to reduce the noise and extract usable infromation from this analysis. This fnction is at the core of our experiment, since it allowed us to draw relevant information from the huge mole of data of the rbg palette extraction.
* the final step is to simply exrtract the 20 most used css color names in each of the datafrmes and then save the information for further use. 





In [ ]:
!pip install webcolors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install colorthief

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from colorthief import ColorThief
from PIL import Image
import glob
Image.MAX_IMAGE_PIXELS = None
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from numpy.random.mtrand import f
from scipy.spatial import KDTree #nearest neighbor search for the closest match in the CSS webcolors dictionary 
from webcolors import (
    CSS3_HEX_TO_NAMES, #able to convert hexval to color name
    hex_to_rgb,
)

def convert_rgb_to_names(rgb_tuple):
    css3_db = CSS3_HEX_TO_NAMES #dictionary mapping hex val to color name 
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return names[index]

In [ ]:
def extract_rgb_palette(adress): 
    colors = []
    for filename in glob.glob(adress):
        color_thief = ColorThief(filename)
        palette = color_thief.get_palette(color_count = 6) #etract the 6 most prominent colors, top quality is default setting 
        colors.append(palette)
    rgb_palette = pd.DataFrame(colors)
    return rgb_palette

def extract_names_palette(rgbPalette): 
    color_names = []
    for index,row in rgbPalette.iterrows(): 
        for rgb_code in row: 
            n = convert_rgb_to_names(rgb_code)
            color_names.append(n)
    names_palette = pd.DataFrame(color_names)
    return names_palette

The extraction pipeline has been perfromed on the most relevant artistic movements. This operation took a while, since some image datasets were very large and many complex operations had to take place. 

The results have been saved as a .csv file in a folder for future usage. 

In [ ]:
#ACADEMIC ART
academicArt_repo = '/content/drive/MyDrive/PAINTINGS_BIG/AcademicArt/*.jpg' 
academicArt_rgb = extract_rgb_palette(academicArt_repo)
academicArt_names = extract_names_palette(academicArt_rgb)

In [ ]:
n = 20
acArt = academicArt_names[0].value_counts()[:n].index.tolist()
acArt = pd.DataFrame(data = acArt)
acArt.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/AcademicArt_PALETTE.csv')

In [ ]:
#POST IMPRESSIONISM
postImp_repo = '/content/drive/MyDrive/PAINTINGS_BIG/PostImpressionism/*.jpg' 
postImp_rgb = extract_rgb_palette(postImp_repo)
postImp_names = extract_names_palette(postImp_rgb)

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [ ]:
postImp = postImp_names[0].value_counts()[:n].index.tolist()
postImp = pd.DataFrame(data = postImp)
postImp.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/postImpressionism_PALETTE.csv')

In [ ]:
#IMPRESSIONISM 
impr_repo = '/content/drive/MyDrive/PAINTINGS_BIG/ImpressionismREDUCED/*.jpg'
impr_rgb = extract_rgb_palette(impr_repo)
impr_names = extract_names_palette(impr_rgb)

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [ ]:
impr = impr_names[0].value_counts()[:n].index.tolist()
impr = pd.DataFrame(data = impr)
impr.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/impressionism_PALETTE.csv')

In [ ]:
#REALISM
realism_repo = '/content/drive/MyDrive/PAINTINGS_BIG/Realism/*.jpg'
realism_rgb = extract_rgb_palette(realism_repo)
realism_names = extract_names_palette(realism_rgb)

In [ ]:
realism = realism_names[0].value_counts()[:n].index.tolist()
realism = pd.DataFrame(data = realism)
realism.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/Realism_PALETTE.csv')

In [ ]:
#ART NOUVEAU 
aNouv_repo = '/content/drive/MyDrive/PAINTINGS_BIG/ArtNouveau/*.jpg'
aNouv_rgb = extract_rgb_palette(aNouv_repo)
aNouv_names = extract_names_palette(aNouv_rgb)

In [ ]:
n=20
artN = aNouv_names[0].value_counts()[:n].index.tolist()
artN = pd.DataFrame(data = artN)
artN.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/ArtNouveau_PALETTE.csv')

In [ ]:
#ORIENTALISM 
orientalism_repo = '/content/drive/MyDrive/PAINTINGS_BIG/Orientalism/*.jpg'
orientalism_rgb = extract_rgb_palette(orientalism_repo)
orientalism_names = extract_names_palette(orientalism_rgb)

In [ ]:
orientalism = orientalism_names[0].value_counts()[:n].index.tolist()
orientalism= pd.DataFrame(data = orientalism)
orientalism.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/Orientalism_PALETTE.csv')

In [ ]:
#NEO IMPRESSIONISM 
neoImp_repo = '/content/drive/MyDrive/PAINTINGS_BIG/NeoImpressionism/*.jpg'
neoImp_rgb = extract_rgb_palette(neoImp_repo)
neoImp_names = extract_names_palette(neoImp_rgb)

In [ ]:
neoImp = neoImp_names[0].value_counts()[:n].index.tolist()
neoImp= pd.DataFrame(data = neoImp)
neoImp.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/NeoImpressionism_PALETTE.csv')

In [ ]:
#EXPRESSIONISM
expressionism_repo = '/content/drive/MyDrive/PAINTINGS_BIG/Expressionism/*.jpg'
expressionism_rgb = extract_rgb_palette(expressionism_repo)
expressionism_names = extract_names_palette(expressionism_rgb)

In [ ]:
expr = expressionism_names[0].value_counts()[:n].index.tolist()
expr= pd.DataFrame(data = expr)
expr.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/Expressionism_PALETTE.csv')

In [ ]:
#MODERNISM
modernism_repo = '/content/drive/MyDrive/PAINTINGS_BIG/Modernism/*.jpg'
modernism_rgb = extract_rgb_palette(modernism_repo)
modernism_names = extract_names_palette(modernism_rgb)

In [ ]:
n = 20 
modernism = modernism_names[0].value_counts()[:n].index.tolist()
modernism= pd.DataFrame(data = modernism)
modernism.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/Modernism_PALETTE.csv')

In [ ]:
#SYMBOLISM 
symbolism_repo = '/content/drive/MyDrive/PAINTINGS_BIG/Symbolism/*.jpg'
symbolism_rgb = extract_rgb_palette(symbolism_repo) 
symbolism_names = extract_names_palette(symbolism_rgb)

In [ ]:
n = 20
symbolism = symbolism_names[0].value_counts()[:n].index.tolist()
symbolism= pd.DataFrame(data = symbolism)
symbolism.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/Symbolism_PALETTE.csv')

In [ ]:
#CUBISM
cubism_repo = '/content/drive/MyDrive/PAINTINGS_BIG/Cubism/*.jpg'
cubism_rgb = extract_rgb_palette(cubism_repo) 
cubism_names = extract_names_palette(cubism_rgb)

In [ ]:
cubism = cubism_names[0].value_counts()[:n].index.tolist()
cubism = pd.DataFrame(data = cubism)
cubism.to_csv('/content/drive/MyDrive/CSV_STORYTELLING_PR/Cubism_PALETTE.csv')

SINGLE IMAGE PALETTE EXTRACTION
(for visualisation purposes)

The following are some interesting experiments in extracting the color palettes from single images, mainly for storytelling and desing purpouses. We used the *extract_rgb_palette* function alone, since in this case a much better accuracy in the color palettes is desirable and useful. 

In [ ]:

lady_with_umbrella = '/content/drive/MyDrive/PAINTINGS_BIG/Impressionism/impressionism4.jpg' #lady with umbrella by Claude Monet
lwu_palette = extract_rgb_palette(lady_with_umbrella)
lwu_palette

,0,1,2,3,4,5
0,"(158, 147, 136)","(115, 97, 57)","(212, 203, 182)","(86, 91, 87)","(81, 72, 80)","(173, 177, 188)"


In [ ]:
la_reverie = '/content/drive/MyDrive/PAINTINGS_BIG/Impressionism/impressionism31.jpg' #La reverie by P.A Renoir 
la_reverie_palette = extract_rgb_palette(la_reverie)
la_reverie_palette

,0,1,2,3,4,5
0,"(182, 149, 119)","(64, 99, 90)","(133, 94, 45)","(113, 65, 48)","(225, 206, 164)","(166, 186, 183)"


In [ ]:
woman_with_flower = '/content/drive/MyDrive/PAINTINGS_BIG/Impressionism/impressionism73.jpg' #lady with flower by Paul Gauguin
wwf_palette = extract_rgb_palette(woman_with_flower)
wwf_palette

,0,1,2,3,4,5
0,"(68, 85, 105)","(233, 196, 85)","(160, 86, 63)","(169, 187, 194)","(128, 153, 185)","(70, 34, 37)"


In [ ]:
adele = '/content/drive/MyDrive/PAINTINGS_BIG/ArtNouveau/artNouveau2.jpg' #portrait of Adele Bloch by Gustav Klimt 
adele_palette = extract_rgb_palette(adele)
adele_palette

,0,1,2,3,4,5
0,"(131, 109, 76)","(232, 202, 136)","(205, 169, 109)","(47, 43, 38)","(183, 176, 174)","(70, 42, 32)"


In [ ]:
hat_w = '/content/drive/MyDrive/PAINTINGS_BIG/ArtNouveau/artNouveau0.jpg' #woman with hat by Gustav Klimt
hat_w_palette = extract_rgb_palette(hat_w)
hat_w_palette

,0,1,2,3,4,5
0,"(195, 144, 105)","(24, 33, 31)","(96, 102, 71)","(42, 65, 96)","(89, 157, 127)","(116, 124, 148)"


In [ ]:
darwin = '/content/drive/MyDrive/PAINTINGS_BIG/AcademicArt/academicArt155.jpg' #darwin portrait by John Collier
darwin_palette = extract_rgb_palette(darwin)
darwin_palette

,0,1,2,3,4,5
0,"(24, 16, 17)","(181, 150, 140)","(103, 75, 66)","(73, 62, 77)","(131, 124, 122)","(129, 108, 132)"


In [ ]:
picasso = '/content/drive/MyDrive/PAINTINGS_BIG/Cubism/cubism0.jpg' #potrait of Picasso by Juan Gris
picasso_palette = extract_rgb_palette(picasso)
picasso_palette

,0,1,2,3,4,5
0,"(151, 152, 147)","(67, 88, 103)","(219, 205, 185)","(221, 219, 213)","(86, 74, 68)","(101, 84, 67)"


In [ ]:
josette = '/content/drive/MyDrive/PAINTINGS_BIG/Cubism/cubism1.jpg' #portrait of Josette by Juan Gris
josette_palette = extract_rgb_palette(josette)
josette_palette

,0,1,2,3,4,5
0,"(199, 185, 183)","(58, 55, 58)","(157, 144, 137)","(171, 172, 177)","(167, 163, 173)","(123, 124, 126)"


In [ ]:
sickGirl = '/content/drive/MyDrive/PAINTINGS_BIG/Realism/realism506.jpg' #sick girl by Christian Krohg
sickGirl_palette = extract_rgb_palette(sickGirl)
sickGirl_palette

,0,1,2,3,4,5
0,"(172, 172, 159)","(41, 42, 22)","(96, 93, 56)","(90, 65, 29)","(104, 111, 94)","(228, 229, 222)"


In [ ]:
lincoln = '/content/drive/MyDrive/PAINTINGS_BIG/Realism/realism197.jpg' #portrait of A. Lincoln by George P. A. Healy 
lincoln_palette = extract_rgb_palette(lincoln)
lincoln_palette

,0,1,2,3,4,5
0,"(12, 6, 5)","(181, 129, 94)","(80, 17, 10)","(124, 124, 124)","(124, 132, 124)","(56, 68, 60)"


In [ ]:
jeanne = '/content/drive/MyDrive/PAINTINGS_BIG/Modernism/modernism80.jpg' #portrait of Jeanne Hébuterne by Amedeo Modigliani
jeanne_palette = extract_rgb_palette(jeanne)
jeanne_palette 

,0,1,2,3,4,5
0,"(187, 135, 78)","(38, 69, 58)","(208, 225, 206)","(95, 127, 107)","(129, 44, 33)","(211, 202, 151)"


In [ ]:
neo = '/content/drive/MyDrive/PAINTINGS_BIG/NeoImpressionism/neoImpr44.jpg'
neo_palette = extract_rgb_palette(neo)
neo_palette

,0,1,2,3,4,5
0,"(190, 148, 122)","(59, 84, 104)","(203, 206, 207)","(215, 135, 34)","(166, 55, 68)","(238, 216, 149)"


In [ ]:
casati = '/content/drive/MyDrive/PAINTINGS_BIG/PostImpressionism/postImp190.jpg' #portrait of Marchesa Luisa Casati by Augustus John
casati_palette = extract_rgb_palette(casati)
casati_palette

,0,1,2,3,4,5
0,"(116, 107, 31)","(229, 208, 124)","(34, 20, 16)","(79, 35, 16)","(184, 182, 104)","(156, 168, 143)"


In [ ]:
expr = '/content/drive/MyDrive/PAINTINGS_BIG/Expressionism/expressionism32.jpg' 
expr_palette = extract_rgb_palette(expr)
expr_palette

,0,1,2,3,4,5
0,"(217, 177, 76)","(74, 78, 78)","(152, 138, 97)","(158, 181, 136)","(86, 114, 144)","(135, 171, 187)"


In [ ]:
nietzsche = '/content/drive/MyDrive/PAINTINGS_BIG/Expressionism/expressionism235.jpg'
nietzsche_palette = extract_rgb_palette(nietzsche)
nietzsche_palette

,0,1,2,3,4,5
0,"(207, 175, 147)","(44, 47, 103)","(100, 91, 129)","(100, 50, 65)","(215, 103, 49)","(149, 98, 95)"


In [ ]:
wally = '/content/drive/MyDrive/PAINTINGS_BIG/Expressionism/expressionism412.jpg'
wally_palette = extract_rgb_palette(wally)
wally_palette

,0,1,2,3,4,5
0,"(34, 40, 33)","(198, 186, 162)","(112, 62, 36)","(146, 104, 62)","(241, 234, 217)","(113, 118, 102)"


In [ ]:
orient = '/content/drive/MyDrive/PAINTINGS_BIG/Orientalism/orientalism42.jpg'
orient_palette = extract_rgb_palette(orient)
orient_palette

,0,1,2,3,4,5
0,"(148, 106, 74)","(236, 233, 218)","(47, 32, 25)","(215, 163, 114)","(219, 196, 135)","(80, 42, 30)"
